In [3]:
import snowflake.connector
import pandas as pd
import numpy as np

In [4]:
ctx = snowflake.connector.connect(
    user='leon.zhao@offerupnow.com',
    password='Steel24ly@offerup',
    account='offerup'
    )
cs = ctx.cursor()
try:
    cs.execute("SELECT current_version()")
    one_row = cs.fetchone()
    print(one_row[0])
finally:
    cs.close()


3.21.2


In [5]:
# Querying Data
results = ctx.cursor().execute("USE WAREHOUSE ANALYTICS_WH").fetchall()
# results = ctx.cursor().execute("USE WAREHOUSE MAINTENANCE").fetchall()

In [31]:
def get_revenue():
    # Querying data by DictCursor
    import pandas as pd
    from snowflake.connector import DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        revenue_data = cur.execute("WITH final AS \
                                   (WIth store_sales_data as \
                                   (SELECT CONVERT_TIMEZONE('America/Los_Angeles', to_timestamp_tz(ORDER_CHARGED_TIMESTAMP))::DATE AS date, \
                                        product_title, \
                                        sku_id AS sku, \
                                        CASE WHEN lower(sku_id) LIKE '%bump%' THEN 'Bumps' \
                                          WHEN lower(sku_id) LIKE '%sellerad%' OR lower(sku_id) LIKE '%feature%' THEN 'Seller Ads a-la-carte' \
                                          WHEN lower(sku_id) LIKE '%subscription.promote%' THEN 'Seller Ads Subscriptions' \
                                          WHEN lower(sku_id) LIKE '%listing_fee%' THEN 'Listing Fees' \
                                          ELSE 'Other' \
                                        END AS purchase_type_group, \
                                        item_price, \
                                        'android' AS platform, \
                                        COUNT(CASE WHEN financial_status = 'Charged' THEN 1 WHEN financial_status = 'Refund' THEN -1 ELSE 0 END) AS units, \
                                        COUNT(CASE WHEN financial_status = 'Charged' THEN 1 ELSE NULL END) AS charged_units, \
                                        COUNT(CASE WHEN financial_status = 'Refund' THEN 1 ELSE NULL END) AS refunded_units, \
                                        SUM(CASE WHEN financial_status = 'Charged' THEN item_price ELSE 0.00 END) AS charged_revenue, \
                                        SUM(CASE WHEN financial_status = 'Refund' THEN item_price ELSE 0.00 END) AS refunded_revenue, \
                                        SUM(CASE WHEN financial_status = 'Charged' THEN item_price WHEN financial_status = 'Refund' THEN -1 * item_price ELSE 0.00 END) AS revenue \
                                  FROM app_stores.public.google_sales \
                                  GROUP BY 1, 2, 3, 4, 5, 6 \
                                 \
                                  UNION ALL \
                                 \
                                  SELECT begin_date AS date, \
                                        title AS product_title, \
                                        sku, \
                                        CASE WHEN lower(sku) LIKE '%bump%' THEN 'Bumps' \
                                          WHEN lower(sku) LIKE '%feature%' OR  lower(sku) LIKE '%sellerad%' THEN 'Seller Ads a-la-carte' \
                                          WHEN lower(sku) LIKE '%subscription.promote%' THEN 'Seller Ads Subscriptions' \
                                          WHEN lower(sku) LIKE '%listing_fee%' THEN 'Listing Fees' \
                                          ELSE 'Other' \
                                        END AS purchase_type_group, \
                                        customer_price AS item_price, \
                                        'iOS' AS platform, \
                                        SUM(CASE WHEN customer_price > 0 THEN units WHEN customer_price < 0 THEN units ELSE 0 END) AS units, \
                                        SUM(CASE WHEN customer_price > 0 THEN units ELSE 0 END) AS charged_units, \
                                        SUM(CASE WHEN customer_price < 0  THEN units ELSE 0 END) AS refunded_units, \
                                        SUM(CASE WHEN customer_price > 0  THEN customer_price * units ELSE 0.00 END) AS charged_revenue, \
                                        SUM(CASE WHEN customer_price < 0  THEN -1 * customer_price * units ELSE 0.00 END) AS refunded_revenue, \
                                        SUM(CASE WHEN customer_price > 0  THEN customer_price * units WHEN customer_price < 0  THEN -1 * customer_price * units ELSE 0.00 END) AS revenue \
                                  FROM app_stores.public.apple_sales_summary \
                                  GROUP BY 1, 2, 3, 4, 5, 6), \
                                  user as \
                                  (WITH first_messages AS \
                                   (select buyer_id, \
                                           min(date_created) as first_message_sent \
                                   from oubi.rds_public.item_itemmessagethread \
                                   group by 1), \
                                   first_posts AS \
                                   (select owner_id, \
                                           min(created) as first_item_posted \
                                   from oubi.rds_public.item_item \
                                   group by 1) \
                                  SELECT u.accepted_tos_id \
                                        ,u.avatar \
                                        ,u.active_changed \
                                        ,u.city \
                                        ,u.created \
                                        ,fp.first_item_posted \
                                        ,fm.first_message_sent \
                                        ,u.gender \
                                        ,u.id \
                                        ,u.modified \
                                        ,u.newsletter_changed \
                                        ,u.payment_enabled \
                                        ,u.payment_enabled_changed \
                                        ,u.seller_account_active \
                                        ,u.seller_account_id \
                                        ,u.seller_approved_or_denied \
                                        ,u.seller_signed_up \
                                        ,u.state \
                                        ,u.user_id \
                                        ,u.verified \
                                        ,u.verified_by_id \
                                        ,u.verified_date \
                                        ,u.welcomed_changed \
                                        ,u.zipcode \
                                        ,case when u.facebook_id is not null then TRUE else FALSE end as facebook_connected \
                                        ,au.is_active \
                                        ,au.username \
                                      from oubi.rds_public.account_userprofile u \
                                      inner join oubi.rds_public.auth_user au on au.id=u.user_id \
                                      left join first_messages fm on fm.buyer_id=u.user_id \
                                      left join first_posts fp on fp.owner_id=u.user_id) \
                                   SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', disbursed_at::timestamp_ntz)::DATE AS date, \
                                       CASE WHEN header_origin = 'android' THEN 'Android' \
                                       WHEN header_origin = 'ios' THEN 'iOS' \
                                       WHEN header_origin = 'iphone' THEN 'iOS' \
                                       WHEN header_origin = 'ipad' THEN 'iOS' \
                                           ELSE header_origin \
                                       END AS platform, \
                                       CASE WHEN city = 'Seattle-Tacoma, WA' OR city = 'Sacramento-Stockton-Modesto, CA' THEN city ELSE 'Other' END AS dimension_1, \
                                       'dma' AS dimension_name, \
                                       'payments_old' AS revenue_source, \
                                       SUM(ou_revenue) AS gross_revenue, \
                                       SUM(payment_amount) AS payment_amount, \
                                       COUNT(DISTINCT item_id::VARCHAR(128) || ' - ' || payer_id::VARCHAR(128)) AS transactions \
                                   FROM \
                                   ( select p.id as id \
                                           ,p.amount as payment_amount \
                                           ,p.commision_amount as ou_revenue \
                                           ,p.settled_at \
                                           ,p.submitted_for_settlement_at \
                                           ,p.offerup_debit_amount \
                                           ,p.buyer_debit_amount \
                                           ,p.fee_collected_from_buyer_amount \
                                           ,p.fee_collected_from_offerup_amount \
                                           ,p.commision_amount \
                                           ,p.commision_rate \
                                           ,oi.item_id \
                                           ,p.created_at \
                                           ,p.authorized_at \
                                           ,p.canceled_at \
                                           ,p.refunded_at \
                                           ,p.disbursed_at \
                                           ,p.canceler_id \
                                           ,p.payment_status \
                                           ,p.payer_id \
                                           ,p.payee_id \
                                           ,p.cancel_reason \
                                           ,pm.card_type \
                                           ,pm.display_name \
                                           ,dma.dma_name as city \
                                           ,pay.header_origin \
                                   from oubi.public.payments_payment p \
                                   left join events_prod.public.PAYMENT_INITIALIZED_EVENT AS pay \
                                       ON p.payer_id = pay.payment_payer_id \
                                           AND p.id = pay.payment_id \
                                   inner join oubi.public.payments_order o on o.id=p.order_id \
                                   inner join oubi.public.payments_orderitem oi on oi.order_id=o.id \
                                   left join oubi.pg.item_dma dma on dma.item_id=oi.item_id \
                                   left join oubi.public.payments_paymentmethod pm on pm.id=p.payment_method_used_id \
                                   where p.payment_gateway_id=2) AS a \
                                   WHERE disbursed_at IS NOT NULL \
                                   GROUP BY 1, 2, 3, 4, 5 \
                                   \
                                   UNION ALL \
                                    \
                                    SELECT date, \
                                    CASE WHEN platform = 'android' THEN 'Android' ELSE platform END AS platform, \
                                    ROUND(item_price,2)::VARCHAR(5) AS dimension_1, \
                                    'price' AS dimension_name, \
                                    'bump' AS revenue_source, \
                                    SUM(revenue) AS gross_revenue, \
                                    0 AS payment_amount, \
                                    SUM(units) AS transactions \
                                    FROM store_sales_data \
                                    WHERE lower(sku) LIKE '%bump%' \
                                    GROUP BY 1, 2, 3 ,4, 5 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT date, \
                                    CASE WHEN platform = 'android' THEN 'Android' ELSE platform END AS platform, \
                                    name AS dimension_1, \
                                    'ad placement' AS dimension_name, \
                                    'external ads' AS revenue_source, \
                                    SUM(CASE WHEN ad_network = 'Bing' THEN estimated_gross_revenue *.69 WHEN ad_network = 'Google' THEN estimated_gross_revenue * .55 ELSE estimated_gross_revenue END) AS gross_revenue, \
                                    0 AS payment_amount, \
                                    SUM(clicks) AS transactions \
                                    FROM \
                                    ( \
                                    SELECT * \
                                    FROM marketing.public.in_app_ads_data \
                                    ) AS a \
                                    GROUP BY 1, 2, 3, 4, 5 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT activity_date AS date, \
                                    CASE WHEN a.ad_unit_name LIKE '%_andr_%' THEN 'Android' \
                                    WHEN a.ad_unit_name LIKE '%_ios_%' THEN 'iOS' \
                                    WHEN a.ad_unit_name LIKE '%web%' THEN 'Web' \
                                    ELSE 'Other' \
                                    END AS platform, \
                                    COALESCE(b.ad_unit_name_lookup, 'Unknown') AS dimension_1, \
                                    'ad placement' AS dimension_name, \
                                    'external ads' AS revenue_source, \
                                    SUM(CASE WHEN activity_date < '2018-03-06' THEN estimated_revenue * 0.69 ELSE estimated_revenue * 0.84 END) AS gross_revenue, \
                                    0 AS payment_amount, \
                                    SUM(clicks) AS transactions \
                                    FROM bing.public.adunit_metrics AS a \
                                    LEFT OUTER JOIN marketing.public.bing_adunit_name_lookup AS b \
                                    ON a.ad_unit_name = b.ad_unit_name \
                                    WHERE activity_date >= '2017-08-01' \
                                    GROUP BY 1, 2, 3, 4, 5 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT date, \
                                    CASE WHEN platform = 'android' THEN 'Android' ELSE platform END AS platform, \
                                    CASE WHEN sku LIKE 'com.offerup.consumer.feature.%' THEN substring(substring(sku, 30, length(sku) - 29), 0, position('.' IN substring(sku, 30, length(sku) - 29))-1) \
                                    WHEN sku LIKE 'com.offerup.iab.feature.%' THEN substring(substring(sku, 25), 0,position('.' IN substring(sku, 25)) - 1) END AS dimension_1, \
                                    'promotion length' AS dimension_name, \
                                    'seller ads a-la-carte' AS revenue_source, \
                                    SUM(revenue) AS gross_revenue, \
                                    0 AS payment_amount, \
                                    SUM(units) AS transactions \
                                    FROM store_sales_data \
                                    WHERE lower(sku) LIKE '%sellerad%' OR lower(sku) LIKE '%feature%' \
                                    GROUP BY 1, 2, 3, 4, 5 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT date, \
                                    CASE WHEN platform = 'android' THEN 'Android' ELSE platform END AS platform, \
                                    CASE WHEN sku LIKE 'com.offerup.consumer.subscription.promote.%' THEN CASE WHEN position('.' IN substring(sku, 43)) = '0' THEN substring(sku, 43, length(sku) - 42) ELSE substring(substring(sku, 43), 0,position('.' IN substring(sku, 43)) - 1) END END AS dimension_1, \
                                    'promotion length' AS dimension_name, \
                                    'seller ads subscriptions' AS revenue_source, \
                                    SUM(revenue) AS gross_revenue, \
                                    0 AS payment_amount, \
                                    SUM(units) AS transactions \
                                    FROM store_sales_data \
                                    WHERE lower(sku) LIKE '%subscription.promote%' \
                                    GROUP BY 1, 2, 3, 4, 5 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT date, \
                                    CASE WHEN platform = 'android' THEN 'Android' ELSE platform END AS platform, \
                                    CASE WHEN sku LIKE 'com.offerup.consumer.listing_fee.%' THEN CASE WHEN position('.' IN substring(sku, 34)) = '0' THEN substring(sku, 34, length(sku) - 33) ELSE substring(substring(sku, 34), 0,position('.' IN substring(sku, 34)) - 1) END END AS dimension_1, \
                                    'listing fee type' AS dimension_name, \
                                    'listing fees' AS revenue_source, \
                                    SUM(revenue) AS gross_revenue, \
                                    0 AS payment_amount, \
                                    SUM(units) AS transactions \
                                    FROM store_sales_data \
                                    WHERE lower(sku) LIKE '%com.offerup.consumer.listing_fee%' \
                                    GROUP BY 1, 2, 3, 4, 5 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT ship_date as date, \
                                    NULL as platform, \
                                    NULL as dimension_1, \
                                    NULL as dimension_name, \
                                    'payments' as revenue_source, \
                                    sum(gross_payment_revenue) as gross_revenue, \
                                    0 as payment_amount, \
                                    count(distinct item_id) as transactions \
                                    FROM marketing.public.shipping_revenue \
                                    GROUP BY 1 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT ship_date as date, \
                                    NULL as platform, \
                                    NULL as dimension_1, \
                                    NULL as dimension_name, \
                                    'shipping' as revenue_source, \
                                    sum(shipping_rev) as gross_revenue, \
                                    0 as payment_amount, \
                                    count(distinct item_id) as transactions \
                                    FROM marketing.public.shipping_revenue \
                                    GROUP BY 1 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', created_at::timestamp_ntz)::DATE AS date, \
                                    CASE WHEN header_origin = 'android' THEN 'Android' \
                                    WHEN header_origin = 'ios' THEN 'iOS' \
                                    WHEN header_origin = 'iphone' THEN 'iOS' \
                                    WHEN header_origin = 'ipad' THEN 'iOS' \
                                    ELSE header_origin \
                                    END AS platform, \
                                    NULL AS dimension_1, \
                                    NULL AS dimension_name, \
                                    'cashless_payments' AS revenue_source, \
                                    SUM(ou_revenue) AS gross_revenue, \
                                    SUM(payment_amount) AS payment_amount, \
                                    COUNT(DISTINCT item_id::VARCHAR(128) || ' - ' || payer_id::VARCHAR(128)) AS transactions \
                                    FROM \
                                    ( \
                                    select p.id as id \
                                    ,p.payer_id \
                                    ,p.created_at \
                                    ,p.payment_status \
                                    ,p.amount as payment_amount \
                                    ,p.commision_amount + p.additional_fees_amount as ou_revenue \
                                    ,pay.header_origin \
                                    ,oi.item_id \
                                    from oubi.public.payments_payment p \
                                    left join events_prod.public.PAYMENT_INITIALIZED_EVENT AS pay \
                                    ON p.payer_id = pay.payment_payer_id \
                                    AND p.id = pay.payment_id \
                                    join oubi.public.payments_orderitem oi \
                                    on oi.order_id = p.order_id \
                                    where payment_source = 'P2P' \
                                    ) AS a \
                                    WHERE payment_status in (5,6,20) \
                                    GROUP BY 1, 2, 3, 4, 5 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', settled_at::timestamp_ntz)::DATE AS date, \
                                    'Web' AS platform, \
                                    payment_amount::VARCHAR(5) AS dimension_1, \
                                    'item price' AS dimension_name, \
                                    'web internal ads purchases' AS revenue_source, \
                                    SUM(payment_amount) AS gross_revenue, \
                                    SUM(payment_amount) AS payment_amount, \
                                    COUNT(1) AS transactions \
                                    FROM \
                                    ( \
                                    select p.id as id \
                                    ,p.amount as payment_amount \
                                    ,p.commision_amount as ou_revenue \
                                    ,p.settled_at \
                                    ,p.submitted_for_settlement_at \
                                    ,p.offerup_debit_amount \
                                    ,p.buyer_debit_amount \
                                    ,p.fee_collected_from_buyer_amount \
                                    ,p.fee_collected_from_offerup_amount \
                                    ,p.commision_amount \
                                    ,p.commision_rate \
                                    ,oi.item_id \
                                    ,p.created_at \
                                    ,p.authorized_at \
                                    ,p.canceled_at \
                                    ,p.refunded_at \
                                    ,p.disbursed_at \
                                    ,p.canceler_id \
                                    ,p.payment_status \
                                    ,p.payer_id \
                                    ,p.payee_id \
                                    ,p.cancel_reason \
                                    ,pm.card_type \
                                    ,pm.display_name \
                                    ,dma.dma_name as city \
                                    ,pay.header_origin \
                                    from oubi.public.payments_payment p \
                                    left join events_prod.public.PAYMENT_INITIALIZED_EVENT AS pay \
                                    ON p.payer_id = pay.payment_payer_id \
                                    AND p.id = pay.payment_id \
                                    inner join oubi.public.payments_order o on o.id=p.order_id \
                                    inner join oubi.public.payments_orderitem oi on oi.order_id=o.id \
                                    left join oubi.pg.item_dma dma on dma.item_id=oi.item_id \
                                    left join oubi.public.payments_paymentmethod pm on pm.id=p.payment_method_used_id \
                                    where payment_source = 'SellerAds' \
                                    ) AS a \
                                    WHERE settled_at IS NOT NULL \
                                    AND canceled_at IS NULL \
                                    AND refunded_at IS NULL \
                                    GROUP BY 1, 2, 3, 4, 5 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT date, \
                                    CASE WHEN header_origin = 'android' THEN 'Android' \
                                    WHEN header_origin = 'ios' THEN 'iOS' \
                                    WHEN header_origin = 'iphone' THEN 'iOS' \
                                    WHEN header_origin = 'ipad' THEN 'iOS' \
                                    ELSE header_origin \
                                    END AS platform, \
                                    payment_price::VARCHAR(4) AS dimension_1, \
                                    'price' AS dimension_name, \
                                    'bump' AS revenue_source, \
                                    SUM(payment_price) AS gross_revenue, \
                                    0 AS payment_amount, \
                                    COUNT(DISTINCT header_unique_id) AS transactions \
                                    FROM \
                                    (SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE AS date, \
                                    ROUND((case when bump_sku='com.offerup.iab.bump' then 0.99 \
                                    when bump_sku='com.offerup.iab.bump.optiontwo' then 1.99 \
                                    when bump_sku='com.offerup.iab.bump.optionthree' then 2.99 \
                                    when bump_sku='com.offerup.iab.bump.optionfour' then 3.99 \
                                    when bump_sku='com.offerup.iab.bump.optionfive' then 4.99 \
                                    when bump_sku='com.offerup.iab.bump.optionsix' then 5.99 \
                                    when bump_sku='com.offerup.iphone.consumer.bump04' then 0.99 \
                                    when bump_sku='com.offerup.iphone.consumer.bump05' then 1.99 \
                                    when bump_sku='com.offerup.iphone.consumer.bump06' then 2.99 \
                                    when bump_sku='com.offerup.iphone.consumer.bump07' then 3.99 \
                                    when bump_sku='com.offerup.iphone.consumer.bump08' then 4.99 \
                                    when bump_sku='com.offerup.iphone.consumer.bump09' then 5.99 \
                                    when header_unique_id='e95e244296bc4685966972d425dcf021' then 3.99 \
                                    when header_device_id='c5cf27a2b15114a4' then 2.99 \
                                    when header_device_id='e8fb7c40c4d435cc' then 1.99 \
                                    else payment_price \
                                    end)::NUMERIC(5,4),2) as payment_price, \
                                    item_id, \
                                    actor_id, \
                                    header_unique_id, \
                                    header_origin \
                                    FROM events_prod.public.item_bumped_event_with_pmt_fix AS item_bumped_event \
                                    LEFT OUTER JOIN user AS seller_info \
                                    ON seller_info.user_id=item_bumped_event.actor_id \
                                    WHERE (item_bumped_event.actor_type <> 'ou_staff' OR item_bumped_event.actor_type IS NULL) \
                                    AND CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE < '2017-10-31' \
                                    AND header_origin IN ('ios', 'iphone', 'ipad') \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE AS date, \
                                    ROUND((case when promo_sku='com.offerup.consumer.bump.tier1.mid.1_99' then 1.99 \
                                    when promo_sku='com.offerup.consumer.bump.tier2.low.3_99' then 3.99 \
                                    else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                                    end)::NUMERIC(7,4),2) as payment_price, \
                                    item_id, \
                                    header_user_id AS actor_id, \
                                    header_unique_id, \
                                    CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                                    WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                                    WHEN promo_sku LIKE '%com.offerup.iab.bump%' THEN 'android' \
                                    ELSE header_origin \
                                    END AS platform \
                                    FROM events_prod.public.item_promoted_event AS a \
                                    WHERE promo_type = 'BUMP' \
                                    AND  CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE < '2017-10-31' \
                                    AND promo_sku LIKE '%com.offerup.consumer%' \
                                    ) AS a \
                                    WHERE payment_price > 0 \
                                    GROUP BY 1, 2, 3 ,4, 5 \
                                    \
                                    UNION ALL \
                                    \
                                    SELECT date, \
                                    CASE WHEN platform = 'android' THEN 'Android' \
                                    WHEN platform = 'ios' THEN 'iOS' \
                                    WHEN platform = 'iphone' THEN 'iOS' \
                                    WHEN platform = 'ipad' THEN 'iOS' \
                                    ELSE platform \
                                    END AS platform, \
                                    LEFT(promo_length, position('D', promo_length) - 1) || 'day' AS dimension_1, \
                                    'promotion length' AS dimension_name, \
                                    'seller ads a-la-carte' AS revenue_source, \
                                    SUM(payment_price) AS gross_revenue, \
                                    0 AS payment_amount, \
                                    COUNT(DISTINCT header_unique_id) AS transactions \
                                    FROM \
                                    (SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE AS date, \
                                    ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                                    when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                                    when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                                    when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                                    when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                                    when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                                    else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                                    end)::NUMERIC(7,4),2) as payment_price, \
                                    item_id, \
                                    header_user_id AS actor_id, \
                                    header_unique_id, \
                                    CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                                    WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                                    WHEN promo_sku LIKE '%com.offerup.iab%' THEN 'android' \
                                    ELSE header_origin \
                                    END AS platform, \
                                    SUBSTRING(promo_type, length('SELLER_AD_') + 1, length(promo_type) - length('SELLER_AD_')) AS promo_length \
                                    FROM events_prod.public.item_promoted_event AS a \
                                    WHERE promo_type <> 'BUMP' \
                                    AND CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE < '2017-10-31' \
                                    AND promo_sku LIKE '%com.offerup.consumer%' \
                                    ) AS a \
                                    GROUP BY 1, 2, 3, 4, 5) \
                                    SELECT date_trunc('month', date) as date_month, \
                                    platform, \
                                    revenue_source, \
                                    round(sum(gross_revenue), 0) as gross_revenue \
                                    FROM final \
                                    WHERE revenue_source in ('seller ads subscriptions', 'bump', 'seller ads a-la-carte', 'external ads', 'web internal ads purchases') \
                                    AND platform in ('iOS', 'Android', 'Web') \
                                    group by 1, 2, 3 \
                                    order by 1 asc, 2 asc ")
        revenue_data = pd.DataFrame(revenue_data.fetchall())
        revenue_data = revenue_data.pivot_table(index=['PLATFORM', 'REVENUE_SOURCE'], values=['GROSS_REVENUE'], columns=['DATE_MONTH'], aggfunc=[np.sum])
        df1 = revenue_data.iloc[-30:]
#         print(df1)
    finally:
        cur.close()
    return revenue_data.to_csv('all_time_revenue.csv')


In [32]:
def get_mau():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        mau_data = cur.execute("SELECT \
                                date_trunc('month', convert_timezone('UTC','America/Los_Angeles',time_utc)) AS month, \
                                CASE WHEN system_name = 'iPhone OS' OR system_name = 'iOS' THEN 'iOS' \
                                WHEN system_name = 'Android OS' THEN 'Android' \
                                ELSE 'Unknown' END AS platform, \
                                count(distinct user_id) AS num \
                                FROM leanplum.derived.leanplum_sessions_is_session \
                                WHERE month >= '2018-01-01' \
                                GROUP BY 1, 2 \
                                ORDER BY 1 ASC, 2 ASC")
        mau_data = pd.DataFrame(mau_data.fetchall())
        mau_data = mau_data.pivot_table(index=['PLATFORM'], values=['NUM'], columns=['MONTH'], aggfunc=[np.sum])
        df1 = mau_data.iloc[-30:]
#         print(mau_data)
    finally:
        cur.close()
    return mau_data.to_csv('updated_mau.csv')


In [4]:
def get_BingExternalAdsData():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        data_bing = cur.execute("SELECT date_trunc('month', activity_date) as month, \
                                case when lower(ad_unit_name) like '%andr%' then 'Android' \
                                when lower(ad_unit_name) like '%ios%' then 'iOS' when lower(AD_UNIT_NAME) like '%web%' then 'Web' else 'Other' end as platform, \
                                sum(clicks)::REAL / nullif(sum(ad_impressions), 0) as ad_ctr, \
                                sum(ad_impressions) / nullif(sum(impressions), 0) as ad_density, \
                                sum(ad_impressions) as ad_impressions, \
                                sum(clicks) as clicks, \
                                SUM(ESTIMATED_REVENUE) / NULLIF(SUM(CLICKS), 0) as CPC, \
                                SUM(CLICKS) / NULLIF(SUM(IMPRESSIONS), 0) as CTR, \
                                SUM(ESTIMATED_REVENUE) as estimated_revenue, \
                                SUM(RAW_SRPV) as Ad_requests, \
                                SUM(IMPRESSIONS) as impressions \
                                FROM bing.public.adunit_metrics \
                                WHERE platform != 'Other' \
                                GROUP BY 1, 2 \
                                ORDER BY 1 asc, 2 desc")
        data_bing = pd.DataFrame(data_bing.fetchall())
        data_bing_1 = data_bing.pivot_table(index=['MONTH'], values=['AD_DENSITY', 'CPC', 'ESTIMATED_REVENUE'], columns=['PLATFORM'], aggfunc=[np.sum])
        data_bing_2 = data_bing.pivot_table(index=['MONTH'], values=['AD_CTR', 'AD_IMPRESSIONS', 'CLICKS'], columns=['PLATFORM'], aggfunc=[np.sum])
        data_bing_3 = data_bing.pivot_table(index=['MONTH'], values=['CTR', 'AD_REQUESTS'], columns=['PLATFORM'], aggfunc=[np.sum])
        df2 = pd.concat([data_bing_1, data_bing_2, data_bing_3], axis=1, sort=False)
        df1 = data_bing.iloc[-30:]
#         print(df2)
    finally:
        cur.close()
    return df2.to_csv('updated_BingExternalAdsData.csv')


In [7]:
def get_BndExternalAdsData():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df2 = []
        data_bnd = cur.execute("SELECT DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp)::DATE) AS month, \
                                case when length(header_device_id) = 36 then 'iOS'  \
                                when (length(header_device_id) = 15 or length(header_device_id) = 16) then 'AD'  \
                                else 'Web' end as platform, \
                                COUNT(1) AS ad_requests, \
                                SUM(number_ads_requested) AS ads_requested, \
                                SUM(CASE WHEN number_ads_returned > number_ads_requested THEN number_ads_requested ELSE number_ads_returned END) AS ads_returned \
                                FROM events_prod.public.materialized_server_ads_requested_event \
                                WHERE header_timestamp >= '2018-01-01' \
                                AND number_ads_requested > 0 \
                                AND ad_request_status = 'true' \
                                AND ad_network = 'bing' \
                                GROUP BY 1, 2")
        data_bnd = pd.DataFrame(data_bnd.fetchall())
        data_bnd = data_bnd.pivot_table(index=['MONTH'], values=['ADS_REQUESTED', 'ADS_RETURNED', 'AD_REQUESTS'], columns=['PLATFORM'], aggfunc=[np.sum])
        df2 = data_bnd.iloc[-30:]
#         print(data_bnd)
    finally:
        cur.close()
    return data_bnd.to_csv('updated_BndExternalAdsData.csv')


In [35]:
def get_SellerAdsData():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        data = cur.execute("WITH active_users_by_month AS ( \
                            SELECT DISTINCT user_id, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', time_utc)::DATE) AS month \
                            FROM leanplum.public.sessions \
                            WHERE is_session = '1' \
                            AND time_utc >= '2016-01-01' \
                            AND TRY_CAST(user_id AS NUMBER) IS NOT NULL \
                            ), active_posts AS ( \
                            SELECT DATE_TRUNC('month',CONVERT_TIMEZONE('America/Los_Angeles', created)::DATE) AS post_month, \
                            DATE_TRUNC('month',CONVERT_TIMEZONE('America/Los_Angeles', sold_date)::DATE) AS sold_month, \
                            DATE_TRUNC('month',CONVERT_TIMEZONE('America/Los_Angeles', archived_date)::DATE) AS archived_month, \
                            review_status, \
                            DATE_TRUNC('month',CONVERT_TIMEZONE('America/Los_Angeles', review_date)::DATE) AS reviewed_month, \
                            owner_id, \
                            id AS item_id \
                            FROM oubi.public.item_item AS b \
                            WHERE priority <> '10000' \
                            ), sub_data AS \
                            (SELECT *, \
                            CONVERT_TIMEZONE('America/Los_Angeles', snapshot_timestamp_str)::DATE AS snapshot_date, \
                            CONVERT_TIMEZONE('America/Los_Angeles', creation_timestamp_str)::DATE AS creation_date, \
                            CONVERT_TIMEZONE('America/Los_Angeles', expiration_timestamp_str)::DATE AS expiration_date, \
                            ROW_NUMBER() OVER(PARTITION BY user_id, purchase_id, platform ORDER BY creation_timestamp_str ASC, expiration_timestamp_str ASC) AS user_purchase_id_num, \
                            ROW_NUMBER() OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC, expiration_timestamp_str ASC) AS user_purchase_num, \
                            DATEDIFF('days', creation_timestamp_str, expiration_timestamp_str) AS days_diff \
                            FROM dynamodb_prod.subscription.subscription_users_log \
                            ), android_free_trials AS \
                            (SELECT *, \
                            MIN(creation_timestamp_str) OVER(PARTITION BY user_id, platform) AS creation_ts, \
                            MIN(expiration_timestamp_str) OVER(PARTITION BY user_id, platform) AS expiration_ts \
                            FROM sub_data \
                            WHERE user_purchase_num = 1 \
                            AND user_purchase_id_num = 1 \
                            AND days_diff <= 5 \
                            AND days_diff >= 3 \
                            AND platform = 'android' \
                            ), ios_free_trials AS \
                            (SELECT DISTINCT user_id, \
                            platform, \
                            MIN(creation_timestamp_str) OVER(PARTITION BY user_id, platform) AS creation_ts, \
                            MIN(expiration_timestamp_str) OVER(PARTITION BY user_id, platform) AS expiration_ts, \
                            FIRST_VALUE(purchase_id) OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC) AS first_purchase_id, \
                            FIRST_VALUE(snapshot_timestamp_str) OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC) AS snapshot_ts \
                            FROM sub_data \
                            WHERE is_current_period_free_trial = 1 \
                            AND platform = 'ios' \
                            ), paid_subs_intermediate AS \
                            (SELECT * \
                            FROM sub_data AS a \
                            WHERE user_purchase_num > 1 \
                            AND days_diff >= 15 \
                            ), paid_sub_prior_purchase AS \
                            (SELECT user_id, \
                            platform, \
                            purchase_id, \
                            LAG(purchase_id) OVER(PARTITION BY user_id, platform ORDER BY min_snapshot_ts ASC) AS prior_purchase_id \
                            FROM \
                            (SELECT user_id, \
                            purchase_id, \
                            platform, \
                            MIN(snapshot_timestamp_str) AS min_snapshot_ts \
                            FROM paid_subs_intermediate \
                            GROUP BY 1, 2, 3 \
                            ) AS a \
                            ), prior_purchase_data AS \
                            (SELECT a.user_id, \
                            a.purchase_id, \
                            a.platform, \
                            MAX(expiration_timestamp_str) AS max_prior_purchase_expiration_timestamp \
                            FROM paid_sub_prior_purchase AS a \
                            JOIN paid_subs_intermediate AS b \
                            ON a.user_id = b.user_id \
                            AND a.platform = b.platform \
                            AND a.prior_purchase_id = b.purchase_id \
                            GROUP BY 1, 2, 3 \
                            ), paid_subs AS \
                            (SELECT a.*, \
                            sum(CASE WHEN purchase_id_num = 1 THEN 1 ELSE 0 END) OVER(PARTITION BY a.user_id, a.platform ORDER BY a.creation_timestamp_str ASC, a.expiration_timestamp_str ASC ROWS UNBOUNDED PRECEDING) AS bill_number \
                            FROM \
                            (SELECT a.*, \
                            b.max_prior_purchase_expiration_timestamp, \
                            ROW_NUMBER() OVER(PARTITION BY a.user_id, a.purchase_id, a.platform ORDER BY a.creation_timestamp_str ASC, a.snapshot_timestamp_str ASC, a.expiration_timestamp_str ASC) AS purchase_id_num \
                            FROM paid_subs_intermediate AS a \
                            LEFT OUTER JOIN prior_purchase_data AS b \
                            ON a.user_id = b.user_id \
                            AND a.platform = b.platform \
                            AND a.purchase_id = b.purchase_id \
                            WHERE DATEDIFF('days', max_prior_purchase_expiration_timestamp, expiration_timestamp_str) >= 15 \
                            OR max_prior_purchase_expiration_timestamp IS NULL) AS a \
                            WHERE purchase_id_num = 1 \
                            ), seller_ads_subscriptions AS \
                            (SELECT user_id, \
                            0 as bill_number, \
                            'android' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_ts::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_ts::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM android_free_trials \
                            GROUP BY 1, 2, 3 \
                            \
                            UNION ALL \
                            \
                            SELECT a.user_id, \
                            bill_number, \
                            'android' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', b.snapshot_ts) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_timestamp_str::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_timestamp_str::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM paid_subs AS a \
                            LEFT OUTER JOIN \
                            (SELECT user_id, \
                            MIN(snapshot_timestamp_str::TIMESTAMP_TZ) AS snapshot_ts \
                            FROM android_free_trials \
                            GROUP BY 1) AS b \
                            ON a.user_id = b.user_id \
                            WHERE platform = 'android' \
                            GROUP BY 1, 2, 3, 4 \
                            \
                            UNION ALL \
                            \
                            SELECT a.user_id, \
                            bill_number, \
                            'ios' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', b.snapshot_ts) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_timestamp_str::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_timestamp_str::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM paid_subs AS a \
                            LEFT OUTER JOIN \
                            (SELECT user_id, \
                            MIN(snapshot_ts::TIMESTAMP_TZ) AS snapshot_ts \
                            FROM ios_free_trials \
                            GROUP BY 1) AS b \
                            ON a.user_id = b.user_id \
                            WHERE platform = 'ios' \
                            GROUP BY 1, 2, 3, 4 \
                            \
                            UNION ALL \
                            \
                            SELECT user_id, \
                            0 AS bill_number, \
                            'ios' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_ts::TIMESTAMP_TZ)) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_ts::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_ts::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_ts::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM ios_free_trials \
                            GROUP BY 1, 2, 3 \
                            ORDER BY user_id ASC \
                            ), item_bumped_and_item_promo AS ( \
                            SELECT actor_id \
                            ,header_user_id \
                            ,actor_type \
                            ,header_user_agent \
                            ,header_app_version \
                            ,header_origin \
                            ,CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp \
                            ,header_unique_id \
                            ,item_id \
                            ,'bump' AS event_name \
                            ,'item_bumped_event' AS table_name \
                            , 'bump' AS promo_type \
                            , 'in_app_purchase' AS paid_source \
                            ,case \
                            when bump_sku='com.offerup.iab.bump' then 0.99 \
                            when bump_sku='com.offerup.iab.bump.optiontwo' then 1.99 \
                            when bump_sku='com.offerup.iab.bump.optionthree' then 2.99 \
                            when bump_sku='com.offerup.iab.bump.optionfour' then 3.99 \
                            when bump_sku='com.offerup.iab.bump.optionfive' then 4.99 \
                            when bump_sku='com.offerup.iab.bump.optionsix' then 5.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump04' then 0.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump05' then 1.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump06' then 2.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump07' then 3.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump08' then 4.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump09' then 5.99 \
                            when header_unique_id='e95e244296bc4685966972d425dcf021' then 3.99 \
                            when header_device_id='c5cf27a2b15114a4' then 2.99 \
                            when header_device_id='e8fb7c40c4d435cc' then 1.99 \
                            when payment_price > 0 THEN payment_price \
                            else 0 \
                            end as payment_price \
                            ,header_device_id \
                            FROM events_prod.public.item_bumped_event_with_pmt_fix \
                            \
                            UNION ALL \
                            \
                            SELECT header_user_id AS actor_id, \
                            header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab.bump%' THEN 'android' \
                            WHEN header_origin = 'ipad' OR header_origin = 'iphone' THEN 'iOS' \
                            ELSE header_origin \
                            END AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            item_id, \
                            'bump' AS event_name, \
                            'item_promoted_event' AS table_name, \
                            promo_type AS promo_type, \
                            paid_source, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            header_device_id \
                            FROM events_prod.public.item_promoted_event \
                            WHERE promo_type = 'BUMP' \
                            \
                            UNION ALL \
                            \
                            SELECT header_user_id AS actor_id, \
                            header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab%' THEN 'android' \
                            WHEN header_origin = 'ipad' OR header_origin = 'iphone' THEN 'iOS' \
                            ELSE header_origin \
                            END AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            CASE WHEN item_id = 'null' THEN NULL ELSE item_id END AS item_id, \
                            'promote' AS event_name, \
                            'item_promoted_event' AS table_name, \
                            promo_type AS promo_type, \
                            paid_source, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            header_device_id \
                            FROM events_prod.public.item_promoted_event \
                            WHERE promo_type <> 'BUMP' \
                            \
                            UNION ALL \
                            \
                            SELECT user_id AS actor_id, \
                            user_id AS header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', snapshot_ts) AS header_timestamp, \
                            user_id || platform || bill_number AS header_unique_id, \
                            NULL AS item_id, \
                            'subscribe' AS event_name, \
                            'subscription_users_log' AS table_name, \
                            'subscribe' AS promo_type, \
                            'subscription purchase' AS paid_source, \
                            19.99 as payment_price, \
                            NULL AS header_device_id \
                            FROM seller_ads_subscriptions \
                            ),step_3 AS ( \
                            SELECT f.month, \
                            f.user_id, \
                            g.platform, \
                            SUM(items) AS items \
                            FROM \
                            (SELECT a.month, \
                            a.user_id, \
                            COUNT(DISTINCT b.item_id) AS items \
                            FROM active_users_by_month AS a \
                            JOIN active_posts AS b \
                            ON a.user_id = b.owner_id::VARCHAR(1776) \
                            AND (post_month <= month AND (sold_month >= month OR sold_month IS NULL) AND (archived_month >= month OR archived_month IS NULL) AND ((review_status = '8' AND reviewed_month >= month) OR review_status <> '8' OR reviewed_month IS NULL)) \
                            GROUP BY 1, 2 \
                            \
                            UNION ALL \
                            \
                            SELECT post_month AS month, \
                            owner_id::VARCHAR(1776) AS user_id, \
                            COUNT(DISTINCT a.item_id) AS items \
                            FROM active_posts AS a \
                            LEFT JOIN active_users_by_month AS b \
                            ON a.owner_id::VARCHAR(1776) = b.user_id \
                            AND b.month = a.post_month \
                            WHERE b.user_id IS NULL \
                            GROUP BY 1, 2 \
                            \
                            UNION ALL \
                            \
                            SELECT DISTINCT a.month, \
                            user_id::VARCHAR(1776) AS user_id, \
                            0 AS items \
                            FROM \
                            (SELECT DISTINCT month FROM active_users_by_month) AS a \
                            LEFT JOIN seller_ads_subscriptions AS b \
                            ON a.month >= DATE_TRUNC('month', b.snapshot_ts::DATE) \
                            AND a.month<= DATE_TRUNC('month', b.expiration_ts::DATE)) AS f \
                            LEFT JOIN \
                            (SELECT a.owner_id, \
                            a.platform, \
                            ROW_NUMBER() OVER(PARTITION BY owner_id ORDER BY posts DESC, last_post DESC) AS posts_rank \
                            FROM \
                            (SELECT owner_id, \
                            CASE WHEN post_with LIKE 'OfferUp/%iOS%' OR post_with LIKE 'OfferUp/%iPhone%' THEN 'iOS' \
                            WHEN post_with LIKE 'OfferUp%Android%' THEN 'Android' \
                            ELSE 'Web' \
                            END AS platform, \
                            MAX(created) AS last_post, \
                            COUNT(1) AS posts \
                            FROM oubi.rds_public.item_item \
                            GROUP BY 1, 2 \
                            ) AS a \
                            ) AS g \
                            ON f.user_id::VARCHAR(1776) = g.owner_id::VARCHAR(1776) \
                            WHERE g.posts_rank = 1 \
                            GROUP BY 1, 2, 3 \
                            ), setp_4 AS \
                            (SELECT DATE_TRUNC('month', header_timestamp::DATE) AS month, \
                            actor_id, \
                            COUNT(DISTINCT item_id) AS items \
                            FROM item_bumped_and_item_promo AS a \
                            WHERE (payment_price > 0 AND (actor_type = 'user' OR actor_type IS NULL)) OR paid_source = 'promo_subscription' \
                            GROUP BY 1 ,2) \
                            SELECT a.month, \
                            a.platform, \
                            COUNT(DISTINCT a.user_id) AS active_sellers, \
                            COUNT(DISTINCT b.actor_id) AS users_purchasing, \
                            SUM(a.items) AS active_items, \
                            SUM(CASE WHEN b.actor_id IS NOT NULL THEN b.items ELSE 0 END) AS active_items_with_purchase_from_advertiser, \
                            SUM(CASE WHEN b.actor_id IS NOT NULL THEN a.items ELSE 0 END) AS active_items_from_purchasers \
                            FROM step_3 a \
                            LEFT JOIN setp_4 b \
                            ON a.month = b.month \
                            AND a.user_id = b.actor_id::VARCHAR(1776) \
                            GROUP BY 1, 2 \
                            ORDER BY 1 ASC, 2 DESC")
        data = pd.DataFrame(data.fetchall())
        data = data.pivot_table(index=['MONTH'], values=['ACTIVE_ITEMS', 'ACTIVE_ITEMS_FROM_PURCHASERS', 'ACTIVE_ITEMS_WITH_PURCHASE_FROM_ADVERTISER', 'ACTIVE_SELLERS', 'USERS_PURCHASING'], columns=['PLATFORM'], aggfunc=[np.sum])
        df1 = data.iloc[-30:]
#         print(data)
    finally:
        cur.close()
    return data.to_csv('updated_SellerAdsData.csv')


In [36]:
def get_IAP_retention():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        data = cur.execute("WITH item_bumped as ( \
                            select actor_id \
                            ,header_user_id \
                            ,actor_type \
                            ,header_user_agent \
                            ,header_app_version \
                            ,header_origin \
                            ,header_timestamp \
                            ,header_unique_id \
                            ,item_id \
                            ,'item_bumped_event' AS event_name \
                            ,case \
                            when bump_sku='com.offerup.iab.bump' then 0.99 \
                            when bump_sku='com.offerup.iab.bump.optiontwo' then 1.99 \
                            when bump_sku='com.offerup.iab.bump.optionthree' then 2.99 \
                            when bump_sku='com.offerup.iab.bump.optionfour' then 3.99 \
                            when bump_sku='com.offerup.iab.bump.optionfive' then 4.99 \
                            when bump_sku='com.offerup.iab.bump.optionsix' then 5.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump04' then 0.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump05' then 1.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump06' then 2.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump07' then 3.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump08' then 4.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump09' then 5.99 \
                            when header_unique_id='e95e244296bc4685966972d425dcf021' then 3.99 \
                            when header_device_id='c5cf27a2b15114a4' then 2.99 \
                            when header_device_id='e8fb7c40c4d435cc' then 1.99 \
                            when payment_price > 0 THEN payment_price \
                            else 0 \
                            end as payment_price \
                            ,header_device_id \
                            from events_prod.public.item_bumped_event_with_pmt_fix \
                            UNION ALL \
                            SELECT header_user_id AS actor_id, \
                            header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            CASE WHEN promo_sku LIKE '%com.offerup.consumer.bump%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab.bump%' THEN 'Android' \
                            ELSE header_origin \
                            END AS header_origin, \
                            header_timestamp, \
                            header_unique_id, \
                            item_id, \
                            'item_promoted_event' AS event_name, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            header_device_id \
                            FROM events_prod.public.item_promoted_event \
                            WHERE promo_type = 'BUMP' \
                            ) ,item_promoted as ( \
                            SELECT header_timestamp, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            item_id, \
                            header_user_id AS actor_id, \
                            header_unique_id, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab%' THEN 'android' \
                            ELSE header_origin \
                            END AS platform, \
                            promo_type, \
                            promo_sku, \
                            CASE WHEN payment_price IS NULL OR payment_price = 0 THEN 'Free' Else 'Paid' END AS promo_price_group, \
                            SUBSTRING(promo_type, length('SELLER_AD_') + 1, length(promo_type) - length('SELLER_AD_')) AS promo_length \
                            FROM events_prod.public.item_promoted_event AS a \
                            WHERE promo_type <> 'BUMP' \
                            ) ,in_app_purchasers AS ( \
                            SELECT DISTINCT month, \
                            week, \
                            platform, \
                            actor_id \
                            FROM ( \
                            select DISTINCT CASE WHEN header_origin IN ('iOS', 'iphone', 'ipad') THEN 'iOS' WHEN header_origin IN ('Android', 'android') THEN 'Android' ELSE 'Other' END AS platform, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp)::DATE)  AS month, \
                            DATE_TRUNC('week', CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp)::DATE) AS week, \
                            actor_id \
                            from item_bumped \
                            WHERE payment_price > 0 \
                            AND ((event_name = 'item_bumped_event' AND actor_type = 'user') OR event_name = 'item_promoted_event') \
                            UNION ALL \
                            select DISTINCT CASE WHEN platform = 'iOS' THEN 'iOS' WHEN platform IN ('android', 'Android') THEN 'Android' ELSE 'Other' END AS platform, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp)::DATE)  AS month, \
                            DATE_TRUNC('week', CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp)::DATE) AS week, \
                            actor_id \
                            from item_promoted \
                            WHERE payment_price > 0) AS a \
                            ), lp_users AS( \
                            SELECT DISTINCT DATE_TRUNC('month', CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', time_utc)::DATE) AS month, \
                            DATE_TRUNC('week', CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', time_utc)::DATE) AS week, \
                            user_id, \
                            CASE WHEN system_name IN ('iOS', 'iPhone OS') THEN 'iOS' WHEN system_name = 'Android OS' THEN 'Android' ELSE 'Other' END AS platform \
                            FROM leanplum.derived.leanplum_sessions_is_session AS a \
                            JOIN in_app_purchasers AS b \
                            ON a.user_id = b.actor_id::VARCHAR(1776) \
                            WHERE time_utc >= '2016-01-01' \
                            ), users_with_active_items AS ( \
                            SELECT a.observation_month, \
                            b.owner_id, \
                            b.platform, \
                            COUNT(DISTINCT CASE WHEN (sold_month >= a.observation_month OR sold_month IS NULL) AND (archived_month >= a.observation_month OR archived_month IS NULL) AND ((review_status = '8' AND reviewed_month >= a.observation_month) OR review_status <> '8') THEN item_id ELSE NULL END) AS items_available_for_sale \
                            FROM( \
                            SELECT DISTINCT \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', created)::DATE) AS observation_month \
                            FROM oubi.public.item_item \
                            ) AS a \
                            JOIN( \
                            SELECT \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', created)::DATE) AS month, \
                            owner_id, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', sold_date)::DATE) AS sold_month, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', archived_date)::DATE) AS archived_month, \
                            review_status, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', review_date)::DATE) AS reviewed_month, \
                            id AS item_id, \
                            CASE WHEN post_with LIKE 'OfferUp/%iOS%' OR post_with LIKE 'OfferUp/%iPhone%' THEN 'iOS' WHEN post_with LIKE 'OfferUp/%Android%' THEN 'Android' ELSE 'Other' END AS platform \
                            FROM oubi.public.item_item) AS b \
                            ON a.observation_month >= b.month \
                            GROUP BY 1, 2, 3 \
                            ), monthly_retained_iapers AS( \
                            SELECT a.month AS time_period, \
                            a.actor_id, \
                            a.platform, \
                            'monthly' AS time_interval, \
                            MAX(CASE WHEN b.actor_id IS NOT NULL THEN 1 ELSE 0 END) AS retained_iaper, \
                            MAX(CASE WHEN (c.user_id IS NOT NULL AND d.owner_id::VARCHAR(1776) IS NOT NULL) OR b.actor_id::VARCHAR(1776) IS NOT NULL THEN 1 ELSE 0 END) AS eligible_retention_iaper \
                            FROM in_app_purchasers AS a \
                            LEFT OUTER JOIN in_app_purchasers AS b \
                            ON a.actor_id = b.actor_id \
                            AND b.month = DATEADD('months', 1, a.month) \
                            AND a.platform = b.platform \
                            LEFT OUTER JOIN lp_users AS c \
                            ON a.actor_id::VARCHAR(1776) = c.user_id \
                            AND c.month = DATEADD('months', 1, a.month) \
                            AND a.platform = c.platform \
                            LEFT OUTER JOIN users_with_active_items AS d \
                            ON d.owner_id = a.actor_id \
                            AND d.observation_month = DATEADD('months', 1, a.month) \
                            AND d.items_available_for_sale > 0 \
                            AND a.platform = d.platform \
                            GROUP BY 1, 2, 3, 4) \
                            SELECT time_period, \
                            dateadd('month', 1, time_period) as actual_month, \
                            platform, \
                            SUM(retained_iaper) as retained_iaper, \
                            SUM(eligible_retention_iaper) as advertisers_returning_from_prior_month \
                            FROM monthly_retained_iapers \
                            GROUP BY 1, 2, 3")
        data = pd.DataFrame(data.fetchall())
        data = data.pivot_table(index=['ACTUAL_MONTH'], values=['RETAINED_IAPER', 'ADVERTISERS_RETURNING_FROM_PRIOR_MONTH'], columns=['PLATFORM'], aggfunc=[np.sum])
        df1 = data.iloc[-30:]
#         print(data)
    finally:
        cur.close()
    return data.to_csv('updated_iapRetentionData.csv')

In [37]:
def get_IAP_retention_total():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        data = cur.execute("WITH item_bumped as ( \
                            select actor_id \
                            ,header_user_id \
                            ,actor_type \
                            ,header_user_agent \
                            ,header_app_version \
                            ,header_origin \
                            ,header_timestamp \
                            ,header_unique_id \
                            ,item_id \
                            ,'item_bumped_event' AS event_name \
                            ,case \
                            when bump_sku='com.offerup.iab.bump' then 0.99 \
                            when bump_sku='com.offerup.iab.bump.optiontwo' then 1.99 \
                            when bump_sku='com.offerup.iab.bump.optionthree' then 2.99 \
                            when bump_sku='com.offerup.iab.bump.optionfour' then 3.99 \
                            when bump_sku='com.offerup.iab.bump.optionfive' then 4.99 \
                            when bump_sku='com.offerup.iab.bump.optionsix' then 5.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump04' then 0.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump05' then 1.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump06' then 2.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump07' then 3.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump08' then 4.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump09' then 5.99 \
                            when header_unique_id='e95e244296bc4685966972d425dcf021' then 3.99 \
                            when header_device_id='c5cf27a2b15114a4' then 2.99 \
                            when header_device_id='e8fb7c40c4d435cc' then 1.99 \
                            when payment_price > 0 THEN payment_price \
                            else 0 \
                            end as payment_price \
                            ,header_device_id \
                            from events_prod.public.item_bumped_event_with_pmt_fix \
                            UNION ALL \
                            SELECT header_user_id AS actor_id, \
                            header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            CASE WHEN promo_sku LIKE '%com.offerup.consumer.bump%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab.bump%' THEN 'Android' \
                            ELSE header_origin \
                            END AS header_origin, \
                            header_timestamp, \
                            header_unique_id, \
                            item_id, \
                            'item_promoted_event' AS event_name, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            header_device_id \
                            FROM events_prod.public.item_promoted_event \
                            WHERE promo_type = 'BUMP' \
                            ) ,item_promoted as ( \
                            SELECT header_timestamp, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            item_id, \
                            header_user_id AS actor_id, \
                            header_unique_id, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab%' THEN 'android' \
                            ELSE header_origin \
                            END AS platform, \
                            promo_type, \
                            promo_sku, \
                            CASE WHEN payment_price IS NULL OR payment_price = 0 THEN 'Free' Else 'Paid' END AS promo_price_group, \
                            SUBSTRING(promo_type, length('SELLER_AD_') + 1, length(promo_type) - length('SELLER_AD_')) AS promo_length \
                            FROM events_prod.public.item_promoted_event AS a \
                            WHERE promo_type <> 'BUMP' \
                            ) ,in_app_purchasers AS ( \
                            SELECT DISTINCT month, \
                            week, \
                            platform, \
                            actor_id \
                            FROM ( \
                            select DISTINCT CASE WHEN header_origin IN ('iOS', 'iphone', 'ipad') THEN 'iOS' WHEN header_origin IN ('Android', 'android') THEN 'Android' ELSE 'Other' END AS platform, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp)::DATE)  AS month, \
                            DATE_TRUNC('week', CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp)::DATE) AS week, \
                            actor_id \
                            from item_bumped \
                            WHERE payment_price > 0 \
                            AND ((event_name = 'item_bumped_event' AND actor_type = 'user') OR event_name = 'item_promoted_event') \
                            UNION ALL \
                            select DISTINCT CASE WHEN platform = 'iOS' THEN 'iOS' WHEN platform IN ('android', 'Android') THEN 'Android' ELSE 'Other' END AS platform, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp)::DATE)  AS month, \
                            DATE_TRUNC('week', CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp)::DATE) AS week, \
                            actor_id \
                            from item_promoted \
                            WHERE payment_price > 0) AS a \
                            ), lp_users AS( \
                            SELECT DISTINCT DATE_TRUNC('month', CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', time_utc)::DATE) AS month, \
                            DATE_TRUNC('week', CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', time_utc)::DATE) AS week, \
                            user_id, \
                            CASE WHEN system_name IN ('iOS', 'iPhone OS') THEN 'iOS' WHEN system_name = 'Android OS' THEN 'Android' ELSE 'Other' END AS platform \
                            FROM leanplum.derived.leanplum_sessions_is_session AS a \
                            JOIN in_app_purchasers AS b \
                            ON a.user_id = b.actor_id::VARCHAR(1776) \
                            WHERE time_utc >= '2016-01-01' \
                            ), users_with_active_items AS ( \
                            SELECT a.observation_month, \
                            b.owner_id, \
                            b.platform, \
                            COUNT(DISTINCT CASE WHEN (sold_month >= a.observation_month OR sold_month IS NULL) AND (archived_month >= a.observation_month OR archived_month IS NULL) AND ((review_status = '8' AND reviewed_month >= a.observation_month) OR review_status <> '8') THEN item_id ELSE NULL END) AS items_available_for_sale \
                            FROM( \
                            SELECT DISTINCT \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', created)::DATE) AS observation_month \
                            FROM oubi.public.item_item \
                            ) AS a \
                            JOIN( \
                            SELECT \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', created)::DATE) AS month, \
                            owner_id, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', sold_date)::DATE) AS sold_month, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', archived_date)::DATE) AS archived_month, \
                            review_status, \
                            DATE_TRUNC('month', CONVERT_TIMEZONE('America/Los_Angeles', review_date)::DATE) AS reviewed_month, \
                            id AS item_id, \
                            CASE WHEN post_with LIKE 'OfferUp/%iOS%' OR post_with LIKE 'OfferUp/%iPhone%' THEN 'iOS' WHEN post_with LIKE 'OfferUp/%Android%' THEN 'Android' ELSE 'Other' END AS platform \
                            FROM oubi.public.item_item) AS b \
                            ON a.observation_month >= b.month \
                            GROUP BY 1, 2, 3 \
                            ), monthly_retained_iapers AS( \
                            SELECT a.month AS time_period, \
                            a.actor_id, \
                            a.platform, \
                            'monthly' AS time_interval, \
                            MAX(CASE WHEN b.actor_id IS NOT NULL THEN 1 ELSE 0 END) AS retained_iaper, \
                            MAX(CASE WHEN (c.user_id IS NOT NULL AND d.owner_id::VARCHAR(1776) IS NOT NULL) OR b.actor_id::VARCHAR(1776) IS NOT NULL THEN 1 ELSE 0 END) AS eligible_retention_iaper \
                            FROM in_app_purchasers AS a \
                            LEFT OUTER JOIN in_app_purchasers AS b \
                            ON a.actor_id = b.actor_id \
                            AND b.month = DATEADD('months', 1, a.month) \
                            AND a.platform = b.platform \
                            LEFT OUTER JOIN lp_users AS c \
                            ON a.actor_id::VARCHAR(1776) = c.user_id \
                            AND c.month = DATEADD('months', 1, a.month) \
                            AND a.platform = c.platform \
                            LEFT OUTER JOIN users_with_active_items AS d \
                            ON d.owner_id = a.actor_id \
                            AND d.observation_month = DATEADD('months', 1, a.month) \
                            AND d.items_available_for_sale > 0 \
                            AND a.platform = d.platform \
                            GROUP BY 1, 2, 3, 4) \
                            SELECT time_period, \
                            dateadd('month', 1, time_period) as actual_month, \
                            SUM(retained_iaper) as retained_iaper, \
                            SUM(eligible_retention_iaper) as advertisers_returning_from_prior_month \
                            FROM monthly_retained_iapers \
                            GROUP BY 1, 2")
        data = pd.DataFrame(data.fetchall())
        data = data.pivot_table(index=['ACTUAL_MONTH'], values=['RETAINED_IAPER', 'ADVERTISERS_RETURNING_FROM_PRIOR_MONTH'], aggfunc=[np.sum])
        df1 = data.iloc[-30:]
#         print(data)
    finally:
        cur.close()
    return data.to_csv('updated_iapRetentionData_total.csv')

In [4]:
def get_itempromo():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        data = cur.execute("with final as( \
                            WITH sub_data AS( \
                            SELECT *, \
                            CONVERT_TIMEZONE('America/Los_Angeles', snapshot_timestamp_str)::DATE AS snapshot_date, \
                            CONVERT_TIMEZONE('America/Los_Angeles', creation_timestamp_str)::DATE AS creation_date, \
                            CONVERT_TIMEZONE('America/Los_Angeles', expiration_timestamp_str)::DATE AS expiration_date, \
                            ROW_NUMBER() OVER(PARTITION BY user_id, purchase_id, platform ORDER BY creation_timestamp_str ASC, expiration_timestamp_str ASC) AS user_purchase_id_num, \
                            ROW_NUMBER() OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC, expiration_timestamp_str ASC) AS user_purchase_num, \
                            DATEDIFF('days', creation_timestamp_str, expiration_timestamp_str) AS days_diff \
                            FROM dynamodb_prod.subscription.subscription_users_log \
                            ), android_free_trials AS \
                            (SELECT *, \
                            MIN(creation_timestamp_str) OVER(PARTITION BY user_id, platform) AS creation_ts, \
                            MIN(expiration_timestamp_str) OVER(PARTITION BY user_id, platform) AS expiration_ts \
                            FROM sub_data \
                            WHERE user_purchase_num = 1 \
                            AND user_purchase_id_num = 1 \
                            AND days_diff <= 5 \
                            AND days_diff >= 3 \
                            AND platform = 'android' \
                            ), ios_free_trials AS \
                            (SELECT DISTINCT user_id, \
                            platform, \
                            MIN(creation_timestamp_str) OVER(PARTITION BY user_id, platform) AS creation_ts, \
                            MIN(expiration_timestamp_str) OVER(PARTITION BY user_id, platform) AS expiration_ts, \
                            FIRST_VALUE(purchase_id) OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC) AS first_purchase_id, \
                            FIRST_VALUE(snapshot_timestamp_str) OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC) AS snapshot_ts \
                            FROM sub_data \
                            WHERE is_current_period_free_trial = 1 \
                            AND platform = 'ios' \
                            ), paid_subs_intermediate AS \
                            (SELECT * \
                            FROM sub_data AS a \
                            WHERE user_purchase_num > 1 \
                            AND days_diff >= 15 \
                            ), paid_sub_prior_purchase AS \
                            (SELECT user_id, \
                            platform, \
                            purchase_id, \
                            LAG(purchase_id) OVER(PARTITION BY user_id, platform ORDER BY min_snapshot_ts ASC) AS prior_purchase_id \
                            FROM \
                            (SELECT user_id, \
                            purchase_id, \
                            platform, \
                            MIN(snapshot_timestamp_str) AS min_snapshot_ts \
                            FROM paid_subs_intermediate \
                            GROUP BY 1, 2, 3 \
                            ) AS a \
                            ), prior_purchase_data AS \
                            (SELECT a.user_id, \
                            a.purchase_id, \
                            a.platform, \
                            MAX(expiration_timestamp_str) AS max_prior_purchase_expiration_timestamp \
                            FROM paid_sub_prior_purchase AS a \
                            JOIN paid_subs_intermediate AS b \
                            ON a.user_id = b.user_id \
                            AND a.platform = b.platform \
                            AND a.prior_purchase_id = b.purchase_id \
                            GROUP BY 1, 2, 3 \
                            ), paid_subs AS \
                            (SELECT a.*, \
                            sum(CASE WHEN purchase_id_num = 1 THEN 1 ELSE 0 END) OVER(PARTITION BY a.user_id, a.platform ORDER BY a.creation_timestamp_str ASC, a.expiration_timestamp_str ASC ROWS UNBOUNDED PRECEDING) AS bill_number \
                            FROM \
                            (SELECT a.*, \
                            b.max_prior_purchase_expiration_timestamp, \
                            ROW_NUMBER() OVER(PARTITION BY a.user_id, a.purchase_id, a.platform ORDER BY a.creation_timestamp_str ASC, a.snapshot_timestamp_str ASC, a.expiration_timestamp_str ASC) AS purchase_id_num \
                            FROM paid_subs_intermediate AS a \
                            LEFT OUTER JOIN prior_purchase_data AS b \
                            ON a.user_id = b.user_id \
                            AND a.platform = b.platform \
                            AND a.purchase_id = b.purchase_id \
                            WHERE DATEDIFF('days', max_prior_purchase_expiration_timestamp, expiration_timestamp_str) >= 15 \
                            OR max_prior_purchase_expiration_timestamp IS NULL) AS a \
                            WHERE purchase_id_num = 1 \
                            ), seller_ads_subscriptions AS \
                            (SELECT user_id, \
                            0 as bill_number, \
                            'android' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_ts::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_ts::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM android_free_trials \
                            GROUP BY 1, 2, 3 \
                            UNION ALL \
                            SELECT a.user_id, \
                            bill_number, \
                            'android' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', b.snapshot_ts) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_timestamp_str::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_timestamp_str::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM paid_subs AS a \
                            LEFT OUTER JOIN \
                            (SELECT user_id, \
                            MIN(snapshot_timestamp_str::TIMESTAMP_TZ) AS snapshot_ts \
                            FROM android_free_trials \
                            GROUP BY 1) AS b \
                            ON a.user_id = b.user_id \
                            WHERE platform = 'android' \
                            GROUP BY 1, 2, 3, 4 \
                            UNION ALL \
                            SELECT a.user_id, \
                            bill_number, \
                            'ios' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', b.snapshot_ts) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_timestamp_str::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_timestamp_str::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM paid_subs AS a \
                            LEFT OUTER JOIN \
                            (SELECT user_id, \
                            MIN(snapshot_ts::TIMESTAMP_TZ) AS snapshot_ts \
                            FROM ios_free_trials \
                            GROUP BY 1) AS b \
                            ON a.user_id = b.user_id \
                            WHERE platform = 'ios' \
                            GROUP BY 1, 2, 3, 4 \
                            UNION ALL \
                            SELECT user_id, \
                            0 AS bill_number, \
                            'ios' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_ts::TIMESTAMP_TZ)) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_ts::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_ts::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_ts::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM ios_free_trials \
                            GROUP BY 1, 2, 3 \
                            ORDER BY user_id ASC) \
                            select actor_id, \
                            header_user_id, \
                            actor_type, \
                            header_user_agent, \
                            header_app_version, \
                            header_origin, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            item_id, \
                            'bump' AS event_name, \
                            'item_bumped_event' AS table_name, \
                            'bump' AS promo_type, \
                            'in_app_purchase' AS paid_source, \
                            case \
                            when bump_sku='com.offerup.iab.bump' then 0.99 \
                            when bump_sku='com.offerup.iab.bump.optiontwo' then 1.99 \
                            when bump_sku='com.offerup.iab.bump.optionthree' then 2.99 \
                            when bump_sku='com.offerup.iab.bump.optionfour' then 3.99 \
                            when bump_sku='com.offerup.iab.bump.optionfive' then 4.99 \
                            when bump_sku='com.offerup.iab.bump.optionsix' then 5.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump04' then 0.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump05' then 1.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump06' then 2.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump07' then 3.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump08' then 4.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump09' then 5.99 \
                            when header_unique_id='e95e244296bc4685966972d425dcf021' then 3.99 \
                            when header_device_id='c5cf27a2b15114a4' then 2.99 \
                            when header_device_id='e8fb7c40c4d435cc' then 1.99 \
                            when payment_price > 0 THEN payment_price \
                            else 0 \
                            end as payment_price, \
                            header_device_id \
                            from events_prod.public.item_bumped_event_with_pmt_fix \
                            UNION ALL \
                            SELECT \
                            header_user_id AS actor_id, \
                            header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab.bump%' THEN 'android' \
                            WHEN header_origin = 'ipad' OR header_origin = 'iphone' THEN 'iOS' \
                            ELSE header_origin \
                            END AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            item_id, \
                            'bump' AS event_name, \
                            'item_promoted_event' AS table_name, \
                            promo_type AS promo_type, \
                            paid_source, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            header_device_id \
                            FROM events_prod.public.item_promoted_event \
                            WHERE promo_type = 'BUMP' \
                            UNION ALL \
                            SELECT \
                            header_user_id AS actor_id, \
                            header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab%' THEN 'android' \
                            WHEN header_origin = 'ipad' OR header_origin = 'iphone' THEN 'iOS' \
                            ELSE header_origin \
                            END AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            CASE WHEN item_id = 'null' THEN NULL ELSE item_id END AS item_id, \
                            'promote' AS event_name, \
                            'item_promoted_event' AS table_name, \
                            promo_type AS promo_type, \
                            paid_source, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            header_device_id \
                            FROM events_prod.public.item_promoted_event \
                            WHERE promo_type <> 'BUMP' \
                            ), first_purchased AS  \
                            (SELECT a.user_id, \
                            MIN(a.first_purchased) AS first_purchased, \
                            SUM(bump_count) AS purchase_count \
                            FROM( \
                            select actor_id as user_id \
                            ,min(cast(header_timestamp as timestamp)) as first_purchased \
                            ,count(*) as bump_count \
                            from events_prod.public.item_bumped_event_with_pmt_fix \
                            where actor_type = 'user' \
                            AND payment_price > 0 \
                            group by 1 \
                            UNION ALL \
                            select header_user_id as user_id \
                            ,min(cast(header_timestamp as timestamp)) as first_purchased \
                            ,count(*) as bump_count \
                            from events_prod.public.item_promoted_event \
                            where promo_type = 'BUMP' \
                            AND promo_price_in_microunits > 0 \
                            group by 1 \
                            UNION ALL \
                            select header_user_id as user_id \
                            ,min(cast(header_timestamp as timestamp)) as first_purchased \
                            ,count(*) as bump_count \
                            from events_prod.public.item_promoted_event \
                            where promo_type <> 'BUMP' \
                            AND promo_price_in_microunits > 0 \
                            group by 1 \
                            ) AS a \
                            GROUP BY 1) \
                            SELECT date_trunc('month', HEADER_TIMESTAMP)::DATE as month, \
                            date_trunc('month', dateadd('month', 1, header_timestamp))::DATE as actual_month, \
                            CASE WHEN header_origin IN ('iOS', 'iPad', 'iphone') THEN 'iOS' WHEN header_origin = 'android' THEN 'Android' WHEN  header_origin LIKE 'web%' THEN 'web' ELSE 'Other' END as platform, \
                            count(1) as Transactions, \
                            count(distinct actor_id) as A_la_carte_Advertisers, \
                            count(distinct case when a.header_timestamp = b.FIRST_PURCHASED then a.actor_id else null end) as first_time_advertisers, \
                            sum(payment_price) as revenue \
                            FROM final a \
                            left join FIRST_PURCHASED b \
                            on a.actor_id = b.user_id \
                            where payment_price > 0 \
                            group by 1, 2, 3 \
                            order by 1, 2, 3 ")
        data = pd.DataFrame(data.fetchall())
        data = data.pivot_table(index=['MONTH'], values=['TRANSACTIONS', 'A_LA_CARTE_ADVERTISERS', 'FIRST_TIME_ADVERTISERS', 'REVENUE'], columns=['PLATFORM'], aggfunc=[np.sum])
        df1 = data.iloc[-30:]
#         print(data)
    finally:
        cur.close()
    return data.to_csv('updated_itempromo.csv')

In [5]:
def get_itempromo_total():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        data = cur.execute("with final as( \
                            WITH sub_data AS( \
                            SELECT *, \
                            CONVERT_TIMEZONE('America/Los_Angeles', snapshot_timestamp_str)::DATE AS snapshot_date, \
                            CONVERT_TIMEZONE('America/Los_Angeles', creation_timestamp_str)::DATE AS creation_date, \
                            CONVERT_TIMEZONE('America/Los_Angeles', expiration_timestamp_str)::DATE AS expiration_date, \
                            ROW_NUMBER() OVER(PARTITION BY user_id, purchase_id, platform ORDER BY creation_timestamp_str ASC, expiration_timestamp_str ASC) AS user_purchase_id_num, \
                            ROW_NUMBER() OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC, expiration_timestamp_str ASC) AS user_purchase_num, \
                            DATEDIFF('days', creation_timestamp_str, expiration_timestamp_str) AS days_diff \
                            FROM dynamodb_prod.subscription.subscription_users_log \
                            ), android_free_trials AS \
                            (SELECT *, \
                            MIN(creation_timestamp_str) OVER(PARTITION BY user_id, platform) AS creation_ts, \
                            MIN(expiration_timestamp_str) OVER(PARTITION BY user_id, platform) AS expiration_ts \
                            FROM sub_data \
                            WHERE user_purchase_num = 1 \
                            AND user_purchase_id_num = 1 \
                            AND days_diff <= 5 \
                            AND days_diff >= 3 \
                            AND platform = 'android' \
                            ), ios_free_trials AS \
                            (SELECT DISTINCT user_id, \
                            platform, \
                            MIN(creation_timestamp_str) OVER(PARTITION BY user_id, platform) AS creation_ts, \
                            MIN(expiration_timestamp_str) OVER(PARTITION BY user_id, platform) AS expiration_ts, \
                            FIRST_VALUE(purchase_id) OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC) AS first_purchase_id, \
                            FIRST_VALUE(snapshot_timestamp_str) OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC) AS snapshot_ts \
                            FROM sub_data \
                            WHERE is_current_period_free_trial = 1 \
                            AND platform = 'ios' \
                            ), paid_subs_intermediate AS \
                            (SELECT * \
                            FROM sub_data AS a \
                            WHERE user_purchase_num > 1 \
                            AND days_diff >= 15 \
                            ), paid_sub_prior_purchase AS \
                            (SELECT user_id, \
                            platform, \
                            purchase_id, \
                            LAG(purchase_id) OVER(PARTITION BY user_id, platform ORDER BY min_snapshot_ts ASC) AS prior_purchase_id \
                            FROM \
                            (SELECT user_id, \
                            purchase_id, \
                            platform, \
                            MIN(snapshot_timestamp_str) AS min_snapshot_ts \
                            FROM paid_subs_intermediate \
                            GROUP BY 1, 2, 3 \
                            ) AS a \
                            ), prior_purchase_data AS \
                            (SELECT a.user_id, \
                            a.purchase_id, \
                            a.platform, \
                            MAX(expiration_timestamp_str) AS max_prior_purchase_expiration_timestamp \
                            FROM paid_sub_prior_purchase AS a \
                            JOIN paid_subs_intermediate AS b \
                            ON a.user_id = b.user_id \
                            AND a.platform = b.platform \
                            AND a.prior_purchase_id = b.purchase_id \
                            GROUP BY 1, 2, 3 \
                            ), paid_subs AS \
                            (SELECT a.*, \
                            sum(CASE WHEN purchase_id_num = 1 THEN 1 ELSE 0 END) OVER(PARTITION BY a.user_id, a.platform ORDER BY a.creation_timestamp_str ASC, a.expiration_timestamp_str ASC ROWS UNBOUNDED PRECEDING) AS bill_number \
                            FROM \
                            (SELECT a.*, \
                            b.max_prior_purchase_expiration_timestamp, \
                            ROW_NUMBER() OVER(PARTITION BY a.user_id, a.purchase_id, a.platform ORDER BY a.creation_timestamp_str ASC, a.snapshot_timestamp_str ASC, a.expiration_timestamp_str ASC) AS purchase_id_num \
                            FROM paid_subs_intermediate AS a \
                            LEFT OUTER JOIN prior_purchase_data AS b \
                            ON a.user_id = b.user_id \
                            AND a.platform = b.platform \
                            AND a.purchase_id = b.purchase_id \
                            WHERE DATEDIFF('days', max_prior_purchase_expiration_timestamp, expiration_timestamp_str) >= 15 \
                            OR max_prior_purchase_expiration_timestamp IS NULL) AS a \
                            WHERE purchase_id_num = 1 \
                            ), seller_ads_subscriptions AS \
                            (SELECT user_id, \
                            0 as bill_number, \
                            'android' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_ts::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_ts::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM android_free_trials \
                            GROUP BY 1, 2, 3 \
                            UNION ALL \
                            SELECT a.user_id, \
                            bill_number, \
                            'android' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', b.snapshot_ts) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_timestamp_str::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_timestamp_str::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM paid_subs AS a \
                            LEFT OUTER JOIN \
                            (SELECT user_id, \
                            MIN(snapshot_timestamp_str::TIMESTAMP_TZ) AS snapshot_ts \
                            FROM android_free_trials \
                            GROUP BY 1) AS b \
                            ON a.user_id = b.user_id \
                            WHERE platform = 'android' \
                            GROUP BY 1, 2, 3, 4 \
                            UNION ALL \
                            SELECT a.user_id, \
                            bill_number, \
                            'ios' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', b.snapshot_ts) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_timestamp_str::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_timestamp_str::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM paid_subs AS a \
                            LEFT OUTER JOIN \
                            (SELECT user_id, \
                            MIN(snapshot_ts::TIMESTAMP_TZ) AS snapshot_ts \
                            FROM ios_free_trials \
                            GROUP BY 1) AS b \
                            ON a.user_id = b.user_id \
                            WHERE platform = 'ios' \
                            GROUP BY 1, 2, 3, 4 \
                            UNION ALL \
                            SELECT user_id, \
                            0 AS bill_number, \
                            'ios' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_ts::TIMESTAMP_TZ)) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_ts::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_ts::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_ts::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM ios_free_trials \
                            GROUP BY 1, 2, 3 \
                            ORDER BY user_id ASC) \
                            select actor_id, \
                            header_user_id, \
                            actor_type, \
                            header_user_agent, \
                            header_app_version, \
                            header_origin, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            item_id, \
                            'bump' AS event_name, \
                            'item_bumped_event' AS table_name, \
                            'bump' AS promo_type, \
                            'in_app_purchase' AS paid_source, \
                            case \
                            when bump_sku='com.offerup.iab.bump' then 0.99 \
                            when bump_sku='com.offerup.iab.bump.optiontwo' then 1.99 \
                            when bump_sku='com.offerup.iab.bump.optionthree' then 2.99 \
                            when bump_sku='com.offerup.iab.bump.optionfour' then 3.99 \
                            when bump_sku='com.offerup.iab.bump.optionfive' then 4.99 \
                            when bump_sku='com.offerup.iab.bump.optionsix' then 5.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump04' then 0.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump05' then 1.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump06' then 2.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump07' then 3.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump08' then 4.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump09' then 5.99 \
                            when header_unique_id='e95e244296bc4685966972d425dcf021' then 3.99 \
                            when header_device_id='c5cf27a2b15114a4' then 2.99 \
                            when header_device_id='e8fb7c40c4d435cc' then 1.99 \
                            when payment_price > 0 THEN payment_price \
                            else 0 \
                            end as payment_price, \
                            header_device_id \
                            from events_prod.public.item_bumped_event_with_pmt_fix \
                            UNION ALL \
                            SELECT \
                            header_user_id AS actor_id, \
                            header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab.bump%' THEN 'android' \
                            WHEN header_origin = 'ipad' OR header_origin = 'iphone' THEN 'iOS' \
                            ELSE header_origin \
                            END AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            item_id, \
                            'bump' AS event_name, \
                            'item_promoted_event' AS table_name, \
                            promo_type AS promo_type, \
                            paid_source, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            header_device_id \
                            FROM events_prod.public.item_promoted_event \
                            WHERE promo_type = 'BUMP' \
                            UNION ALL \
                            SELECT \
                            header_user_id AS actor_id, \
                            header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab%' THEN 'android' \
                            WHEN header_origin = 'ipad' OR header_origin = 'iphone' THEN 'iOS' \
                            ELSE header_origin \
                            END AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            CASE WHEN item_id = 'null' THEN NULL ELSE item_id END AS item_id, \
                            'promote' AS event_name, \
                            'item_promoted_event' AS table_name, \
                            promo_type AS promo_type, \
                            paid_source, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            header_device_id \
                            FROM events_prod.public.item_promoted_event \
                            WHERE promo_type <> 'BUMP' \
                            ), first_purchased AS  \
                            (SELECT a.user_id, \
                            MIN(a.first_purchased) AS first_purchased, \
                            SUM(bump_count) AS purchase_count \
                            FROM( \
                            select actor_id as user_id \
                            ,min(cast(header_timestamp as timestamp)) as first_purchased \
                            ,count(*) as bump_count \
                            from events_prod.public.item_bumped_event_with_pmt_fix \
                            where actor_type = 'user' \
                            AND payment_price > 0 \
                            group by 1 \
                            UNION ALL \
                            select header_user_id as user_id \
                            ,min(cast(header_timestamp as timestamp)) as first_purchased \
                            ,count(*) as bump_count \
                            from events_prod.public.item_promoted_event \
                            where promo_type = 'BUMP' \
                            AND promo_price_in_microunits > 0 \
                            group by 1 \
                            UNION ALL \
                            select header_user_id as user_id \
                            ,min(cast(header_timestamp as timestamp)) as first_purchased \
                            ,count(*) as bump_count \
                            from events_prod.public.item_promoted_event \
                            where promo_type <> 'BUMP' \
                            AND promo_price_in_microunits > 0 \
                            group by 1 \
                            ) AS a \
                            GROUP BY 1) \
                            SELECT date_trunc('month', HEADER_TIMESTAMP)::DATE as month, \
                            date_trunc('month', dateadd('month', 1, header_timestamp))::DATE as actual_month, \
                            count(1) as Transactions, \
                            count(distinct actor_id) as A_la_carte_Advertisers, \
                            count(distinct case when a.header_timestamp = b.FIRST_PURCHASED then a.actor_id else null end) as first_time_advertisers, \
                            sum(payment_price) as revenue \
                            FROM final a \
                            left join FIRST_PURCHASED b \
                            on a.actor_id = b.user_id \
                            where payment_price > 0 \
                            group by 1, 2 \
                            order by 1, 2 ")
        data = pd.DataFrame(data.fetchall())
        data = data.pivot_table(index=['MONTH'], values=['TRANSACTIONS', 'A_LA_CARTE_ADVERTISERS', 'FIRST_TIME_ADVERTISERS', 'REVENUE'], aggfunc=[np.sum])
        df1 = data.iloc[-30:]
#         print(data)
    finally:
        cur.close()
    return data.to_csv('updated_itempromo_total.csv')

In [6]:
def get_itempromo_breakdown():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        data = cur.execute("with final as( \
                            WITH sub_data AS( \
                            SELECT *, \
                            CONVERT_TIMEZONE('America/Los_Angeles', snapshot_timestamp_str)::DATE AS snapshot_date, \
                            CONVERT_TIMEZONE('America/Los_Angeles', creation_timestamp_str)::DATE AS creation_date, \
                            CONVERT_TIMEZONE('America/Los_Angeles', expiration_timestamp_str)::DATE AS expiration_date, \
                            ROW_NUMBER() OVER(PARTITION BY user_id, purchase_id, platform ORDER BY creation_timestamp_str ASC, expiration_timestamp_str ASC) AS user_purchase_id_num, \
                            ROW_NUMBER() OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC, expiration_timestamp_str ASC) AS user_purchase_num, \
                            DATEDIFF('days', creation_timestamp_str, expiration_timestamp_str) AS days_diff \
                            FROM dynamodb_prod.subscription.subscription_users_log \
                            ), android_free_trials AS \
                            (SELECT *, \
                            MIN(creation_timestamp_str) OVER(PARTITION BY user_id, platform) AS creation_ts, \
                            MIN(expiration_timestamp_str) OVER(PARTITION BY user_id, platform) AS expiration_ts \
                            FROM sub_data \
                            WHERE user_purchase_num = 1 \
                            AND user_purchase_id_num = 1 \
                            AND days_diff <= 5 \
                            AND days_diff >= 3 \
                            AND platform = 'android' \
                            ), ios_free_trials AS \
                            (SELECT DISTINCT user_id, \
                            platform, \
                            MIN(creation_timestamp_str) OVER(PARTITION BY user_id, platform) AS creation_ts, \
                            MIN(expiration_timestamp_str) OVER(PARTITION BY user_id, platform) AS expiration_ts, \
                            FIRST_VALUE(purchase_id) OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC) AS first_purchase_id, \
                            FIRST_VALUE(snapshot_timestamp_str) OVER(PARTITION BY user_id, platform ORDER BY creation_timestamp_str ASC) AS snapshot_ts \
                            FROM sub_data \
                            WHERE is_current_period_free_trial = 1 \
                            AND platform = 'ios' \
                            ), paid_subs_intermediate AS \
                            (SELECT * \
                            FROM sub_data AS a \
                            WHERE user_purchase_num > 1 \
                            AND days_diff >= 15 \
                            ), paid_sub_prior_purchase AS \
                            (SELECT user_id, \
                            platform, \
                            purchase_id, \
                            LAG(purchase_id) OVER(PARTITION BY user_id, platform ORDER BY min_snapshot_ts ASC) AS prior_purchase_id \
                            FROM \
                            (SELECT user_id, \
                            purchase_id, \
                            platform, \
                            MIN(snapshot_timestamp_str) AS min_snapshot_ts \
                            FROM paid_subs_intermediate \
                            GROUP BY 1, 2, 3 \
                            ) AS a \
                            ), prior_purchase_data AS \
                            (SELECT a.user_id, \
                            a.purchase_id, \
                            a.platform, \
                            MAX(expiration_timestamp_str) AS max_prior_purchase_expiration_timestamp \
                            FROM paid_sub_prior_purchase AS a \
                            JOIN paid_subs_intermediate AS b \
                            ON a.user_id = b.user_id \
                            AND a.platform = b.platform \
                            AND a.prior_purchase_id = b.purchase_id \
                            GROUP BY 1, 2, 3 \
                            ), paid_subs AS \
                            (SELECT a.*, \
                            sum(CASE WHEN purchase_id_num = 1 THEN 1 ELSE 0 END) OVER(PARTITION BY a.user_id, a.platform ORDER BY a.creation_timestamp_str ASC, a.expiration_timestamp_str ASC ROWS UNBOUNDED PRECEDING) AS bill_number \
                            FROM \
                            (SELECT a.*, \
                            b.max_prior_purchase_expiration_timestamp, \
                            ROW_NUMBER() OVER(PARTITION BY a.user_id, a.purchase_id, a.platform ORDER BY a.creation_timestamp_str ASC, a.snapshot_timestamp_str ASC, a.expiration_timestamp_str ASC) AS purchase_id_num \
                            FROM paid_subs_intermediate AS a \
                            LEFT OUTER JOIN prior_purchase_data AS b \
                            ON a.user_id = b.user_id \
                            AND a.platform = b.platform \
                            AND a.purchase_id = b.purchase_id \
                            WHERE DATEDIFF('days', max_prior_purchase_expiration_timestamp, expiration_timestamp_str) >= 15 \
                            OR max_prior_purchase_expiration_timestamp IS NULL) AS a \
                            WHERE purchase_id_num = 1 \
                            ), seller_ads_subscriptions AS \
                            (SELECT user_id, \
                            0 as bill_number, \
                            'android' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_ts::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_ts::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM android_free_trials \
                            GROUP BY 1, 2, 3 \
                            UNION ALL \
                            SELECT a.user_id, \
                            bill_number, \
                            'android' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', b.snapshot_ts) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_timestamp_str::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_timestamp_str::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM paid_subs AS a \
                            LEFT OUTER JOIN \
                            (SELECT user_id, \
                            MIN(snapshot_timestamp_str::TIMESTAMP_TZ) AS snapshot_ts \
                            FROM android_free_trials \
                            GROUP BY 1) AS b \
                            ON a.user_id = b.user_id \
                            WHERE platform = 'android' \
                            GROUP BY 1, 2, 3, 4 \
                            UNION ALL \
                            SELECT a.user_id, \
                            bill_number, \
                            'ios' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', b.snapshot_ts) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_timestamp_str::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_timestamp_str::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_timestamp_str::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM paid_subs AS a \
                            LEFT OUTER JOIN \
                            (SELECT user_id, \
                            MIN(snapshot_ts::TIMESTAMP_TZ) AS snapshot_ts \
                            FROM ios_free_trials \
                            GROUP BY 1) AS b \
                            ON a.user_id = b.user_id \
                            WHERE platform = 'ios' \
                            GROUP BY 1, 2, 3, 4 \
                            UNION ALL \
                            SELECT user_id, \
                            0 AS bill_number, \
                            'ios' AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_ts::TIMESTAMP_TZ)) AS trial_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(creation_ts::TIMESTAMP_TZ)) AS creation_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(expiration_ts::TIMESTAMP_TZ)) AS expiration_ts, \
                            CONVERT_TIMEZONE('America/Los_Angeles', MIN(snapshot_ts::TIMESTAMP_TZ)) AS snapshot_ts \
                            FROM ios_free_trials \
                            GROUP BY 1, 2, 3 \
                            ORDER BY user_id ASC) \
                            select actor_id, \
                            header_user_id, \
                            actor_type, \
                            header_user_agent, \
                            header_app_version, \
                            header_origin, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            item_id, \
                            'bump' AS event_name, \
                            'item_bumped_event' AS table_name, \
                            'bump' AS promo_type, \
                            'in_app_purchase' AS paid_source, \
                            case \
                            when bump_sku='com.offerup.iab.bump' then 0.99 \
                            when bump_sku='com.offerup.iab.bump.optiontwo' then 1.99 \
                            when bump_sku='com.offerup.iab.bump.optionthree' then 2.99 \
                            when bump_sku='com.offerup.iab.bump.optionfour' then 3.99 \
                            when bump_sku='com.offerup.iab.bump.optionfive' then 4.99 \
                            when bump_sku='com.offerup.iab.bump.optionsix' then 5.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump04' then 0.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump05' then 1.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump06' then 2.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump07' then 3.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump08' then 4.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump09' then 5.99 \
                            when header_unique_id='e95e244296bc4685966972d425dcf021' then 3.99 \
                            when header_device_id='c5cf27a2b15114a4' then 2.99 \
                            when header_device_id='e8fb7c40c4d435cc' then 1.99 \
                            when payment_price > 0 THEN payment_price \
                            else 0 \
                            end as payment_price, \
                            header_device_id \
                            from events_prod.public.item_bumped_event_with_pmt_fix \
                            UNION ALL \
                            SELECT \
                            header_user_id AS actor_id, \
                            header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab.bump%' THEN 'android' \
                            WHEN header_origin = 'ipad' OR header_origin = 'iphone' THEN 'iOS' \
                            ELSE header_origin \
                            END AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            item_id, \
                            'bump' AS event_name, \
                            'item_promoted_event' AS table_name, \
                            promo_type AS promo_type, \
                            paid_source, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            header_device_id \
                            FROM events_prod.public.item_promoted_event \
                            WHERE promo_type = 'BUMP' \
                            UNION ALL \
                            SELECT \
                            header_user_id AS actor_id, \
                            header_user_id, \
                            NULL AS actor_type, \
                            NULL AS header_user_agent, \
                            NULL AS header_app_version, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab%' THEN 'android' \
                            WHEN header_origin = 'ipad' OR header_origin = 'iphone' THEN 'iOS' \
                            ELSE header_origin \
                            END AS platform, \
                            CONVERT_TIMEZONE('America/Los_Angeles', header_timestamp) AS header_timestamp, \
                            header_unique_id, \
                            CASE WHEN item_id = 'null' THEN NULL ELSE item_id END AS item_id, \
                            'promote' AS event_name, \
                            'item_promoted_event' AS table_name, \
                            promo_type AS promo_type, \
                            paid_source, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            header_device_id \
                            FROM events_prod.public.item_promoted_event \
                            WHERE promo_type <> 'BUMP' \
                            ), first_purchased AS  \
                            (SELECT a.user_id, \
                            MIN(a.first_purchased) AS first_purchased, \
                            SUM(bump_count) AS purchase_count \
                            FROM( \
                            select actor_id as user_id \
                            ,min(cast(header_timestamp as timestamp)) as first_purchased \
                            ,count(*) as bump_count \
                            from events_prod.public.item_bumped_event_with_pmt_fix \
                            where actor_type = 'user' \
                            AND payment_price > 0 \
                            group by 1 \
                            UNION ALL \
                            select header_user_id as user_id \
                            ,min(cast(header_timestamp as timestamp)) as first_purchased \
                            ,count(*) as bump_count \
                            from events_prod.public.item_promoted_event \
                            where promo_type = 'BUMP' \
                            AND promo_price_in_microunits > 0 \
                            group by 1 \
                            UNION ALL \
                            select header_user_id as user_id \
                            ,min(cast(header_timestamp as timestamp)) as first_purchased \
                            ,count(*) as bump_count \
                            from events_prod.public.item_promoted_event \
                            where promo_type <> 'BUMP' \
                            AND promo_price_in_microunits > 0 \
                            group by 1 \
                            ) AS a \
                            GROUP BY 1) \
                            SELECT date_trunc('month', HEADER_TIMESTAMP)::DATE as month, \
                            date_trunc('month', dateadd('month', 1, header_timestamp))::DATE as actual_month, \
                            CASE WHEN header_origin IN ('iOS', 'iPad', 'iphone') THEN 'iOS' WHEN header_origin = 'android' THEN 'Android' WHEN  header_origin LIKE 'web%' THEN 'web' ELSE 'Other' END as platform, \
                            CASE WHEN promo_type in ('bump', 'BUMP') THEN 'BUMP' ELSE promo_type END AS promo_type, \
                            count(1) as Transactions, \
                            count(distinct actor_id) as A_la_carte_Advertisers, \
                            count(distinct case when a.header_timestamp = b.FIRST_PURCHASED then a.actor_id else null end) as first_time_advertisers, \
                            sum(payment_price) as revenue \
                            FROM final a \
                            left join FIRST_PURCHASED b \
                            on a.actor_id = b.user_id \
                            where payment_price > 0 \
                            and promo_type not in ('MUST_GO') \
                            group by 1, 2, 3, 4 \
                            order by 1, 2, 3, 4 ")
        data = pd.DataFrame(data.fetchall())
        data = data.pivot_table(index=['MONTH'], values=['REVENUE', 'TRANSACTIONS', 'A_LA_CARTE_ADVERTISERS', 'FIRST_TIME_ADVERTISERS'], columns=['PLATFORM', 'PROMO_TYPE'], aggfunc=[np.sum])
        df1 = data.iloc[-30:]
#         print(data)
    finally:
        cur.close()
    return data.to_csv('updated_itempromo_breakdown.csv')

In [46]:
def get_subTnC():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        mau_data = cur.execute("SELECT trunc(event_time_utc, 'month')::DATE AS DATE, \
                                case when SYSTEM_NAME = 'Android OS' then 'AD' else 'iOS' end as PLATFORM, \
                                count(case when event_name IN ('UI_Event') and EVENT_PAR_SCREEN_NAME IN ('subscription_terms_and_conditions') then user_id else null end) as subscription_tnc_Screenviews, \
                                count(distinct case when event_name IN ('UI_Event') and EVENT_PAR_SCREEN_NAME IN ('subscription_terms_and_conditions') then user_id else null end) as subscription_tnc_Users \
                                FROM leanplum.public.events \
                                WHERE event_name IN ('UI_Event') \
                                and EVENT_PAR_SCREEN_NAME IN ('subscription_terms_and_conditions') \
                                and event_time_utc >= '2018-01-01' \
                                GROUP BY 1, 2 \
                                ORDER BY 1, 2")
        mau_data = pd.DataFrame(mau_data.fetchall())
        mau_data = mau_data.pivot_table(index=['DATE'], values=['SUBSCRIPTION_TNC_SCREENVIEWS', 'SUBSCRIPTION_TNC_USERS'], columns=['PLATFORM'], aggfunc=[np.sum])
        df1 = mau_data.iloc[-30:]
#         print(mau_data)
    finally:
        cur.close()
    return mau_data.to_csv('updated_subTnC.csv')

In [1]:
def get_promo_clicked_buy():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        mau_data = cur.execute("SELECT trunc(event_time_utc, 'month')::DATE AS DATE, \
                                case when SYSTEM_NAME = 'Android OS' then 'AD' else 'iOS' end as PLATFORM, \
                                case when event_name IN ('itempromo_clicked_buy') then event_parameters:promo_type else null end as promo_type_clicked, \
                                count(1) as events_clicked, \
                                count(distinct user_id) as users_clicked \
                                FROM leanplum.public.events \
                                WHERE event_name IN ('itempromo_clicked_buy') \
                                and event_time_utc >= '2018-01-01' \
                                and promo_type_clicked is not null \
                                GROUP BY 1, 2, 3 \
                                ORDER BY 1, 2, 3")
        mau_data = pd.DataFrame(mau_data.fetchall())
        mau_data = mau_data.pivot_table(index=['DATE'], values=['EVENTS_CLICKED', 'USERS_CLICKED'], columns=['PLATFORM', 'PROMO_TYPE_CLICKED'], aggfunc=[np.sum])
        df1 = mau_data.iloc[-30:]
#         print(mau_data)
    finally:
        cur.close()
    return mau_data.to_csv('updated_promo_clicked_buy.csv')

In [8]:
def get_promo_screen():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        data = cur.execute("SELECT trunc(event_time_utc, 'month')::DATE AS DATE, \
                            case when SYSTEM_NAME = 'Android OS' then 'AD' else 'iOS' end as PLATFORM, \
                            count(case when event_name IN ('screenview') and EVENT_PAR_SCREEN_NAME IN ('ItemPromoSelection') then user_id else null end) as ItemPromoSelection_Screenviews, \
                            count(distinct case when event_name IN ('screenview') and EVENT_PAR_SCREEN_NAME IN ('ItemPromoSelection') then user_id else null end) as ItemPromoSelection_Users \
                            FROM leanplum.public.event_screenview \
                            WHERE (event_name IN ('screenview') and EVENT_PAR_SCREEN_NAME IN ('ItemPromoSelection')) \
                            AND event_time_utc >= '2018-01-01' \
                            GROUP BY 1, 2 \
                            ORDER BY 1, 2")
        data = pd.DataFrame(data.fetchall())
        data = data.pivot_table(index=['DATE'], values=['ITEMPROMOSELECTION_SCREENVIEWS', 'ITEMPROMOSELECTION_USERS'], columns=['PLATFORM'], aggfunc=[np.sum])
        df1 = data.iloc[-30:]
    finally:
        cur.close()
    return data.to_csv('updated_promo_screen.csv')

In [33]:
def get_cvt_source():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        data = cur.execute("SELECT * FROM ou_users.leon.sell_faster_cvt_source")
        data = pd.DataFrame(data.fetchall())
        data = data.pivot_table(index=['DATE'], values=['ITEMPERFORMANCE_VIEWED', 'EDITITEM3_VIEWED', 'MYOFFERSSELLING_VIEWED', 'ITEMDASHBOARD_VIEWED', 'POSTITEMCONFIRMATION_VIEWED', 'EDITITEMCONFIRMATION_VIEWED'], columns=['SYSTEM_NAME'], aggfunc=[np.sum])
        df1 = data.iloc[-30:]
    finally:
        cur.close()
    return data.to_csv('updated_cvt_source.csv')







In [53]:
def get_two():
    try:
        get_subTnC()
        get_promo_clicked_buy()
        get_promo_screen()
    except:
        print('HOLY COW! WE HAVE AN ISSUE!')


In [7]:
get_itempromo()
get_itempromo_total()
get_itempromo_breakdown()

In [34]:
get_cvt_source()

In [51]:
def get_mbr():
    try:
        get_revenue()
        get_mau()
        get_BingExternalAdsData()
        get_BndExternalAdsData()
        get_SellerAdsData()
        get_IAP_retention()
        get_IAP_retention_total()
        get_itempromo()
        get_itempromo_total()
        get_subTnC()
        get_promo_clicked_buy()
        get_promo_screen()
    except:
        print('HOLY COW! WE HAVE AN ISSUE!')


In [ ]:
get_BndExternalAdsData()

In [ ]:
ctx.close()